In [1]:

import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2 

DataDir="F:/2022_Summer/data"
Categories=["Post-Injury", "Pre-Injury"]

trainingdata=[] 
def createtrainingdata():
#creates a data set of images and labels 
    print("Training data is loading...")
    for category in Categories: 
        path=os.path.join(DataDir,category)
        classnum=Categories.index(category)
        for vid in os.listdir(path):
            vpath=os.path.join(path,vid)
            #vid_array=[]
            for frame in os.listdir(vpath): 
                img_array=cv2.imread(os.path.join(vpath,frame),0)
               #vid_array.append(img_array)
                trainingdata.append([img_array,classnum])
            
    print("Done!")
createtrainingdata()

Training data is loading...
Done!


In [2]:
#Randomizes the order of the data set 
import random 
random.shuffle(trainingdata)
print("Data is randomized!")
for sample in trainingdata[:10]:
    print(sample[1])

Data is randomized!
0
1
1
0
0
0
0
0
0
0


In [8]:
#Changes the traingdata into X and Y numpy arrays to be inputted into the model 
X=[]
Y=[]
for features,label in trainingdata:
    X.append(features)
    Y.append(label)

X=np.array(X)
Y=np.array(Y)
X=X.reshape(-1,X[1].shape[0],X[1].shape[1],1)
print("Data is formatted!")

Data is formatted!


In [4]:
#saves training data 
import pickle 
pickle_out=open("X.pickle", "wb")
pickle.dump(X,pickle_out)
pickle_out.close()
pickle_out=open("Y.pickle", "wb")
pickle.dump(Y,pickle_out)
pickle_out.close()

In [10]:

import tensorflow as tf 
import pickle 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time 

NAME="Pre-vs-Post-Injury-CNN-{}".format(int(time.time()))

tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))

X=pickle.load(open("X.pickle", "rb"))
Y=pickle.load(open("Y.pickle", "rb"))
#Normalizes the values 
X=X/255.0

model=Sequential()
model.add(Conv2D(64, (3,3),input_shape=X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dense(1))

model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

model.fit(X,Y, batch_size=32,epochs=3,validation_split=0.2,callbacks=[tensorboard])


Epoch 1/3
1383/1383 [==============================] - 42s 30ms/step - loss: 0.0179 - accuracy: 0.9941 - val_loss: 1.2202e-06 - val_accuracy: 1.0000
Epoch 2/3
1383/1383 [==============================] - 39s 28ms/step - loss: 3.2786e-07 - accuracy: 1.0000 - val_loss: 1.6525e-07 - val_accuracy: 1.0000
Epoch 3/3
1383/1383 [==============================] - 39s 28ms/step - loss: 8.8965e-08 - accuracy: 1.0000 - val_loss: 6.4031e-08 - val_accuracy: 1.0000


ValueError: in user code:

    File "C:\Users\DeepLabCut\anaconda3\envs\vidan\lib\site-packages\keras\engine\training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\DeepLabCut\anaconda3\envs\vidan\lib\site-packages\keras\engine\training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\DeepLabCut\anaconda3\envs\vidan\lib\site-packages\keras\engine\training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\DeepLabCut\anaconda3\envs\vidan\lib\site-packages\keras\engine\training.py", line 1791, in predict_step
        return self(x, training=False)
    File "C:\Users\DeepLabCut\anaconda3\envs\vidan\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "conv2d_2" (type Conv2D).
    

In [17]:

print(int(model.predict(X[:1]))
print(Y[0])

SyntaxError: '(' was never closed (2404721033.py, line 1)